In [0]:
%sql
Create TABLE datamodelling_vasu.default.scdtype1_source
(
  prod_id INT,
  prod_name STRING,
  prod_category STRING,
  process_date DATE
)

In [0]:
%sql
Insert into datamodelling_vasu.default.scdtype1_source
values
(1,'prod1','category1',current_date()),
(2,'prod2','category2',current_date()),
(3,'prod3','category3',current_date()),
(4,'prod4','category4',current_date()),
(5,'prod5','category5',current_date());

In [0]:
%sql
update datamodelling_vasu.default.scdtype1_source
set prod_category = 'new_category'
where prod_id = 3;

In [0]:
%sql
create table datamodelling_vasu.gold.scdtype1_table
(
  prod_id INT,
  prod_name STRING,
  prod_category STRING,
  process_date DATE
)

In [0]:
%sql
Merge into datamodelling_vasu.gold.scdtype1_table t
using datamodelling_vasu.default.scdtype1_source s
on t.prod_id = s.prod_id
when matched and s.process_date >= t.process_date then update set *  --back dated refresh should not affect the latest changes on the records
when not matched then insert *;

In [0]:
%sql
select * from datamodelling_vasu.gold.scdtype1_table

### SCD TYPE 2

In [0]:
%sql
Create or replace TABLE datamodelling_vasu.default.scdtype2_source
(
  prod_id INT,
  prod_name STRING,
  prod_category STRING,
  process_date DATE
)

In [0]:
%sql
Insert into datamodelling_vasu.default.scdtype2_source
values
(1,'prod1','category1',current_date()),
(2,'prod2','category2',current_date()),
(3,'prod3','category3',current_date()),
(4,'prod4','category4',current_date()),
(5,'prod5','category5',current_date());

In [0]:
%sql
select * from datamodelling_vasu.default.scdtype2_source

In [0]:
%sql
CREATE OR REPLACE TABLE datamodelling_vasu.gold.scdtype2_table
(
  prod_id INT,
  prod_name STRING,
  prod_category STRING,
  process_date DATE,
  start_date DATE,
  end_date DATE,
  current_flag Boolean
)

In [0]:
%sql
--set spark.sql.session.timezone = America/Toronto
CREATE OR REPLACE TEMPORARY VIEW srctype2
AS
SELECT *,
    date_format(from_utc_timestamp(current_timestamp(), 'America/Toronto'),'MM-dd-yyyy HH:mm:ss') as start_date,
    date_format(CAST('3000-01-01 00:00:00' AS timestamp),'MM-dd-yyyy HH:mm:ss') as end_date,
    true as current_flag
FROM datamodelling_vasu.default.scdtype2_source

In [0]:
spark.sql("Select * from datamodelling_vasu.default.scdtype2_source").createOrReplaceTempView("srctype2")

In [0]:
%sql
select * from srctype2

In [0]:
%sql
update datamodelling_vasu.default.scdtype2_source
set prod_category = 'new_category'
where prod_id = 3;

In [0]:
%sql
MERGE INTO datamodelling_vasu.gold.scdtype2_table trg
USING srctype2 src
ON trg.prod_id = src.prod_id
AND trg.current_flag = true
-- WHEN WE HAVE A MATCH AND THE RECORD IS UPDATED THEN UPDATE THE RECORD
WHEN MATCHED AND (
  src.prod_name <> trg.prod_name OR
  src.prod_category <> trg.prod_category OR
  src.process_date <> trg.process_date)
THEN UPDATE SET 
  trg.end_date = TRY_CAST(from_utc_timestamp(current_timestamp(), 'America/Toronto') AS timestamp),
  trg.current_flag = false
WHEN NOT MATCHED THEN INSERT *